In [1]:
# Sample API
import flowx
import matplotlib.pyplot as plt
from scipy.interpolate import griddata 
import numpy
import sys
#%matplotlib notebook
flowx.__version__

'0.1.dev'

In [2]:
# Define grid parameters
nx, ny = 129, 129
xmin, xmax = 0.0, 1.0
ymin, ymax = 0.0, 1.0

# Re   = 100.0
# dt   = 0.01
# tmax = 10.

# Define cell-centered variable names
center_vars = ['pres', 'divv', 'asol', 'eror']
face_vars   = ['velc', 'hvar', 'asol', 'eror']

# Define boundary conditions for variable pressure and velocity [left, right, bottom, top]
bc_type_pres = {'pres': ['neumann', 'neumann', 'neumann', 'neumann']}
bc_val_pres  = {'pres': [0.0, 0.0, 0.0, 0.0]}

bc_type_u = {'velc': ['dirichlet', 'dirichlet', 'dirichlet', 'dirichlet']}
bc_val_u  = {'velc': [0.0, 0.0, 0.0, 1.0]}

bc_type_v = {'velc': ['dirichlet', 'dirichlet', 'dirichlet', 'dirichlet']}
bc_val_v  = {'velc': [0.0, 0.0, 0.0, 0.0]}

# Create the grid and data
gridc = flowx.Grid('cell-centered', center_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_pres, user_bc_val=bc_val_pres)

gridx = flowx.Grid('x-face', face_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_u, user_bc_val=bc_val_u)

gridy = flowx.Grid('y-face', face_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_v, user_bc_val=bc_val_v)

scalars = flowx.Scalars(tmax=10.0, dt=0.0001, Re=50.0)

In [ ]:
ins_vars = ['velc', 'hvar', 'divv', 'pres']

poisson_options = dict(maxiter = 2000, tol = 1e-9)
poisson_options['lu'], poisson_options['mtx'] = flowx.poisson.build_sparse(gridc, 'pres')

# 1st iteration with Euler explicit scheme

# Predictor Step
flowx.ins.advance_euler(gridc, gridx, gridy, scalars, ins_vars, 'predictor')

# Divergence Step
flowx.ins.advance_euler(gridc, gridx, gridy, scalars, ins_vars, 'divergence')

# Solve the pressure Poisson equation 
scalars.stats['ites'], scalars.stats['res'] = flowx.poisson.solve_lu(gridc, 'pres', 'divv', 
                                                                                    poisson_options)
# Corrector Step
flowx.ins.advance_euler(gridc, gridx, gridy, scalars, ins_vars, 'corrector')

# Display stats
if scalars.variable['nstep'] % 10 == 0:
        flowx.io.display_stats(scalars)   

scalars.advance()


while scalars.variable['time'] <= scalars.variable['tmax']:
    
    # Predictor Step
    flowx.ins.ab2(gridc, gridx, gridy, scalars, ins_vars, 'predictor')

    # Divergence Step
    flowx.ins.ab2(gridc, gridx, gridy, scalars, ins_vars, 'divergence')
    
    # Solve the pressure Poisson equation 
    scalars.stats['ites'], scalars.stats['res'] = flowx.poisson.solve_lu(gridc, 'pres', 'divv', 
                                                                             poisson_options)
    
    # Corrector Step
    flowx.ins.ab2(gridc, gridx, gridy, scalars, ins_vars, 'corrector')
    
    # Display stats
    if scalars.variable['nstep'] % 10 == 0:
        flowx.io.display_stats(scalars)   

    scalars.advance()

------------ Time = 0.0 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.342797132779488e-09
Max, Min, U   : 2.0, -0.013165308862013931
Max, Min, V   : 0.03327762279104301, -0.03327762279104301
Max, Min, P   : 15.253546478869376, -15.935457593872917
Max, Min, DIV : 2.070565940925917e-14, -2.0755966390062497e-14


------------ Time = 0.0010000000000000002 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.3466066066717308e-09
Max, Min, U   : 2.0, -0.06673057222115074
Max, Min, V   : 0.19628556092283586, -0.19628556092283586
Max, Min, P   : 10.776295711988432, -11.105407896284067
Max, Min, DIV : 1.1546319456101628e-14, -1.164346397075633e-14


------------ Time = 0.0020000000000000005 ---------------
Number of poisson iterations    : None
Final poisson residual : 9.73519319633556e-10
Max, Min, U   : 2.0, -0.09182134757054557
Max, Min, V   : 0.24732674617725053, -0.24732674617725053
Max, Min, P   : 9.12861912806709, -9.151950

------------ Time = 0.02399999999999991 ---------------
Number of poisson iterations    : None
Final poisson residual : 3.118090011957054e-10
Max, Min, U   : 2.0, -0.13609459012981232
Max, Min, V   : 0.3260003698710693, -0.3556729276738265
Max, Min, P   : 6.564936147786374, -6.184634991621001
Max, Min, DIV : 1.2864709297844001e-14, -1.3360840211973368e-14


------------ Time = 0.024999999999999904 ---------------
Number of poisson iterations    : None
Final poisson residual : 3.105961262945723e-10
Max, Min, U   : 2.0, -0.1362890226107911
Max, Min, V   : 0.3265809544456312, -0.3567066728345518
Max, Min, P   : 6.546187613645432, -6.174568550634298
Max, Min, DIV : 1.3197776205231548e-14, -1.3135326160096383e-14


------------ Time = 0.025999999999999898 ---------------
Number of poisson iterations    : None
Final poisson residual : 3.04944757703373e-10
Max, Min, U   : 2.0, -0.13693281957443124
Max, Min, V   : 0.32709908931378523, -0.3576532156612518
Max, Min, P   : 6.526436453606632, -6.1

------------ Time = 0.04800000000000035 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.3824070429853635e-10
Max, Min, U   : 2.0, -0.14176370734249602
Max, Min, V   : 0.3402734183075162, -0.37896350739019774
Max, Min, P   : 6.535244881740299, -5.832446239971171
Max, Min, DIV : 1.27675647831893e-14, -1.3766765505351941e-14


------------ Time = 0.04900000000000038 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.3321466934000865e-10
Max, Min, U   : 2.0, -0.14207208030001559
Max, Min, V   : 0.3405505029813592, -0.37954538767489904
Max, Min, P   : 6.5402574210349345, -5.819067795618243
Max, Min, DIV : 1.27675647831893e-14, -1.3489209749195652e-14


------------ Time = 0.050000000000000405 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.3136851223004187e-10
Max, Min, U   : 2.0, -0.14233191317049293
Max, Min, V   : 0.34081152569687817, -0.380102844214923
Max, Min, P   : 6.4746428861595495, -

------------ Time = 0.07200000000000104 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.094940048543035e-10
Max, Min, U   : 2.0, -0.14577106544456267
Max, Min, V   : 0.3440250115248523, -0.3882704207670136
Max, Min, P   : 6.477971757846243, -5.7497904611529345
Max, Min, DIV : 1.3111040031432708e-14, -1.3406810384086754e-14


------------ Time = 0.07300000000000106 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.031483108849137e-10
Max, Min, U   : 2.0, -0.1458524170328789
Max, Min, V   : 0.34410040239750256, -0.3885178712843868
Max, Min, P   : 6.407306238799854, -5.816528273714314
Max, Min, DIV : 1.359329315775426e-14, -1.353648096391602e-14


------------ Time = 0.07400000000000109 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.197726029458569e-10
Max, Min, U   : 2.0, -0.14591219875685893
Max, Min, V   : 0.3441720843890477, -0.3887581464794586
Max, Min, P   : 6.390189193438195, -5.829

------------ Time = 0.09600000000000172 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.9015459290812962e-10
Max, Min, U   : 2.0, -0.14872966418488975
Max, Min, V   : 0.3451036984742461, -0.3942817158472306
Max, Min, P   : 6.433672274863092, -5.721853997773833
Max, Min, DIV : 1.27675647831893e-14, -1.3655743202889425e-14


------------ Time = 0.09700000000000175 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.0256799033697146e-10
Max, Min, U   : 2.0, -0.14885044704972414
Max, Min, V   : 0.34512612337147786, -0.394634951625021
Max, Min, P   : 6.438028281429495, -5.715251814618891
Max, Min, DIV : 1.4210854715202004e-14, -1.3350431871117507e-14


------------ Time = 0.09800000000000178 ---------------
Number of poisson iterations    : None
Final poisson residual : 2.075598559212718e-10
Max, Min, U   : 2.0, -0.14903107036044733
Max, Min, V   : 0.3451473926921993, -0.39498102156747966
Max, Min, P   : 6.3849152730370555, -5.

...

------------ Time = 4.833000000002075 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.5458553099229294e-10
Max, Min, U   : 2.0, -0.21921993717149518
Max, Min, V   : 0.34361399089366845, -0.46692116021837893
Max, Min, P   : 6.451975508366516, -5.561021081747924
Max, Min, DIV : 1.3655743202889425e-14, -1.3766765505351941e-14


------------ Time = 4.834000000002073 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.539262758943762e-10
Max, Min, U   : 2.0, -0.21921992192371753
Max, Min, V   : 0.34361398570087437, -0.46692133137726366
Max, Min, P   : 6.450674954524454, -5.562321887923509
Max, Min, DIV : 1.454392162258955e-14, -1.362104873336989e-14


------------ Time = 4.83500000000207 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.660193517883312e-10
Max, Min, U   : 2.0, -0.21921990668371272
Max, Min, V   : 0.3436139805135073, -0.46692150234984536
Max, Min, P   : 6.470413242028231, -5.54258

------------ Time = 4.859000000002014 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.7233513341135097e-10
Max, Min, U   : 2.0, -0.21921954329917634
Max, Min, V   : 0.3436138576303479, -0.46692555033446254
Max, Min, P   : 6.456315377890538, -5.556687686332443
Max, Min, DIV : 1.4127587988355117e-14, -1.4654943925052066e-14


------------ Time = 4.860000000002012 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.6447013245960737e-10
Max, Min, U   : 2.0, -0.21921952825889693
Max, Min, V   : 0.34361385257685406, -0.4669257167159922
Max, Min, P   : 6.471065082058882, -5.5419382276052715
Max, Min, DIV : 1.3766765505351941e-14, -1.3655743202889425e-14


------------ Time = 4.86100000000201 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.4946614401627545e-10
Max, Min, U   : 2.0, -0.21921951322681296
Max, Min, V   : 0.3436138475286437, -0.4669258829165133
Max, Min, P   : 6.494931009676719, -5.518

------------ Time = 4.883000000001958 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.6627542981820473e-10
Max, Min, U   : 2.0, -0.21921918462423196
Max, Min, V   : 0.3436137377937921, -0.46692949393620115
Max, Min, P   : 6.453550718544171, -5.55945816469479
Max, Min, DIV : 1.4210854715202004e-14, -1.354472090042691e-14


------------ Time = 4.884000000001956 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.655050206993356e-10
Max, Min, U   : 2.0, -0.21921916978466469
Max, Min, V   : 0.34361373286561187, -0.46692965602840164
Max, Min, P   : 6.460339346229588, -5.552669776254102
Max, Min, DIV : 1.3433698597964394e-14, -1.3211653993039363e-14


------------ Time = 4.885000000001954 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.5254154139336042e-10
Max, Min, U   : 2.0, -0.21921915495363484
Max, Min, V   : 0.34361372794258604, -0.46692981794434274
Max, Min, P   : 6.45816328944863, -5.5548

------------ Time = 4.9070000000019025 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.5784866265322602e-10
Max, Min, U   : 2.0, -0.2192188308570845
Max, Min, V   : 0.3436136209293876, -0.46693333589385433
Max, Min, P   : 6.476817151418825, -5.536197403911538
Max, Min, DIV : 1.5987211554602254e-14, -1.4210854715202004e-14


------------ Time = 4.9080000000019 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.6659576674314475e-10
Max, Min, U   : 2.0, -0.21921881622593808
Max, Min, V   : 0.3436136161234577, -0.4669334938092165
Max, Min, P   : 6.463344633709255, -5.5496701548244465
Max, Min, DIV : 1.3100631690576847e-14, -1.3988810110276972e-14


------------ Time = 4.909000000001898 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.6326851032844003e-10
Max, Min, U   : 2.0, -0.21921880160362828
Max, Min, V   : 0.3436136113225562, -0.46693365155293964
Max, Min, P   : 6.533454695348796, -5.4795

------------ Time = 4.932000000001844 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.6683855671528855e-10
Max, Min, U   : 2.0, -0.219218467755177
Max, Min, V   : 0.3436135022777931, -0.4669372327199675
Max, Min, P   : 6.501640831550822, -5.511379479936798
Max, Min, DIV : 1.4016565685892601e-14, -1.3877787807814457e-14


------------ Time = 4.933000000001842 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.593317777444229e-10
Max, Min, U   : 2.0, -0.21921845334826398
Max, Min, V   : 0.343613497596029, -0.4669373864007928
Max, Min, P   : 6.423932532181568, -5.589088006376973
Max, Min, DIV : 1.3711254354120683e-14, -1.3988810110276972e-14


------------ Time = 4.93400000000184 ---------------
Number of poisson iterations    : None
Final poisson residual : 1.8474574697097159e-10
Max, Min, U   : 2.0, -0.2192184389504569
Max, Min, V   : 0.34361349291916515, -0.4669375399146591
Max, Min, P   : 6.4147385541310555, -5.598282211

In [ ]:
flowx.io.plot_contour(gridc, 'pres')
flowx.io.plot_contour(gridx, 'velc')
flowx.io.plot_vector(gridx,gridy,'velc')

In [ ]:
u = gridx.get_values('velc')
v = gridy.get_values('velc')

omega_u = (-u[:,1:] + u[:,:-1]) / gridx.dx
omega_v = (v[1:,:] - v[:-1,:]) / gridx.dx
omega = omega_u + omega_v

contour = plt.contour(-omega.transpose(), levels = [-3,-2,-1,-0.5,0,0.5,1,2,3,4,5])
plt.clabel(contour, inline=1, fontsize=10 , colors =  'k')
#plt.colorbar(extend='both')
plt.clim(-3,5)
plt.axis('scaled')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# Define meshgrid 
Xfx, Yfx = numpy.meshgrid(gridx.x, gridx.y)
Xfy, Yfy = numpy.meshgrid(gridy.x, gridy.y)

Xfx, Yfx = Xfx.transpose(), Yfx.transpose()
Xfy, Yfy = Xfy.transpose(), Yfy.transpose()

Xc, Yc = numpy.meshgrid(gridc.x, gridc.y)
Xc, Yc = Xc.transpose(), Yc.transpose()

# flatten for u
Xfx = Xfx.flatten(order='C')
Yfx = Yfx.flatten(order='C')
u = u.flatten(order='C')

#flatten for v
Xfy = Xfy.flatten(order='C')
Yfy = Yfy.flatten(order='C')
v = v.flatten(order='C')

# Interpolation u at gridc
uc = numpy.zeros(numpy.shape(u))
uc = griddata((Xfx, Yfx), u, (Xc, Yc), method='nearest')

# Interpolation v at gridc
vc = numpy.zeros(numpy.shape(v))
vc = griddata((Xfy, Yfy), v, (Xc, Yc), method='nearest')

In [ ]:
streamlines = plt.streamplot(gridc.x, gridc.y, uc.transpose(), vc.transpose(), color = 'tab:blue', density = 2, linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.autoscale(enable=True, axis='y', tight=True)
plt.axis('scaled')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
# Load csv files
x_a = numpy.loadtxt("a.csv")[:,0]
y_a = numpy.loadtxt("a.csv")[:,1]

x_b = numpy.loadtxt("b.csv")[:,0]
y_b = numpy.loadtxt("b.csv")[:,1]

In [ ]:
plt.plot(gridx.get_values('velc')[65,:],gridy.x, 'b', label = 'Present Study')
plt.plot(x_a, y_a, 'r', label = 'Ghia et. al')
plt.autoscale(enable=True, axis='x', tight=True)
plt.autoscale(enable=True, axis='y', tight=True)
plt.axis('scaled')
plt.legend(fontsize = 13)
plt.xlabel('u')
plt.ylabel('y')

In [ ]:
plt.plot(gridy.x,gridy.get_values('velc')[:,int(ny/2)],label = 'Present Study')
plt.plot(x_b, y_b, 'r', label = 'Ghia et. al')
plt.autoscale(enable=True, axis='x', tight=True)
plt.autoscale(enable=True, axis='y', tight=True)
#plt.axis('scaled')
plt.legend(fontsize = 13)
plt.xlabel('x')
plt.ylabel('v')